# Seed words finder for PMI score computation
Disclaymer: To run this notebook, launch pyspark (command "pyspark --master local[*number of cores*]") from the folder containing the notebook.

In [1]:
from pyspark.sql import SparkSession
from utility_functions import *
import os

In [2]:
# Create spark session
spark = SparkSession.builder \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "32g") \
    .config("spark.network.timeout", "1200s") \
    .config("spark.executor.memoryOverhead", "12g")\
    .config("spark.executor.heartbeatInterval", "1200s")\
    .config("spark.executor.extraJavaOptions", "-Xmx32g -Xms12g") \
    .getOrCreate()

In [3]:
# Get directory
directory = os.path.dirname(os.getcwd()).replace("\\", "/")

# Define paths
path = directory + "/data/datasets/dataset-cleaned-no-unknown.json"
path_csv = directory + "/data/sentiment-knowledge/words-seed-score.csv"

In [4]:
# Load dataset
dataset = spark.read.json(path, schema="overall float, reviewText string")

In [5]:
save_list_to_csv(dataset.rdd.flatMap(lambda obj: [(word, calculate_polarity_of_occurrence(obj["overall"])) for word in tokenize_with_sequences(remove_symbols_before_tokenization(obj["reviewText"]))])\
            .reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1]).collect(), path_csv, ["word", "score"])